In [1]:
import geopandas as gpd
import json
import spacy
import tqdm
import pandas as pd
import re
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
from ipywidgets import interact

PATH_TO_FEEDS = 'Feeds/*.jsonl'

nlp = spacy.load("en_core_web_lg") #natural language processing model
nlp.disable_pipes("tagger", "parser", "ner")

[('tagger', <spacy.pipeline.pipes.Tagger at 0x11af36fd0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x11cc5a768>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x11cc5a7c8>)]

## Load Feeds

In [2]:
feed_addresses = glob.glob(PATH_TO_FEEDS)
feed_addresses = feed_addresses[1:]
print(feed_addresses)

['Feeds/IQOS_search_result_2019-07-31_11_31_21.097108.jsonl', 'Feeds/Heat_Not_Burn_search_result_2019-07-24_07_48_28.067985.jsonl', 'Feeds/Vaping_search_result_2019-07-24_07_48_25.721746.jsonl']


In [3]:
desired_stances = ["POSITIVE_sent_probas","POSITIVE_sent_probas","POSITIVE_sent_probas"] #desired stance for every feed - Must be manually input by user
assert len(desired_stances) == len(feed_addresses)


In [4]:
feeds = [] # 2d-list of feeds with articles of each
total_articles = 0
for i, adress in enumerate(feed_addresses):
    feeds.append([])
    for line in open(adress):
        feeds[i].append(json.loads(line))
        total_articles +=1


CHANCE_TO_REPLACE_SNIPPET = 3.0 / total_articles  # percent chance for a current snippet to replace a stored one
print("Loaded", total_articles, "documents")
print(len(feeds))

Loaded 13431 documents
3


## Setup

In [12]:
harm_reduction_keywords = ["harm", "reduce", "alternative", "safe", "healthy", "help", "quit", "switch"]
#ensure keywords are in fact properly lemmanized
for i in range(len(harm_reduction_keywords)):
    harm_reduction_keywords[i] = nlp(harm_reduction_keywords[i])[0].lemma_.lower()
    
    
harm_reduction_by_country = {}
    
def add_to_countries(country, year):
    if not country in harm_reduction_by_country:
        harm_reduction_by_country[country] = {}
        for yr in range(1990,2020):
            harm_reduction_by_country[country][str(yr)] = 0
    else:
        harm_reduction_by_country[country][year] += 1
            
def format_date(string):
    return string[:4]    


def avg_sentence_sentiment(article,desired_stance):
    temp = 0.0
    for prob in article["_source"][desired_stance]:
        if(prob != -1):
            temp += prob
    return temp / len(article["_source"][desired_stance])

## Iterate Over Data

In [13]:
THRESHOLD = .1

YEAR_THRESHOLD = 1990

#iterate over feeds
for feed_number, feed in enumerate(feeds):
    #iterate over the articles
    for article in tqdm.tqdm_notebook(feed):
        date = format_date(article["_source"]["date"])
        
        #1. check > 1990
        if int(date) > YEAR_THRESHOLD:
        
            #2. check for pro-industry
            if avg_sentence_sentiment(article,desired_stances[feed_number]) >= THRESHOLD:#article["_source"][desired_stances[feed_number]] > THRESHOLD:
                doc = nlp(article["_source"]["text"])


                #3. check for harm reduction keyword
                for i, word in enumerate(doc):
                    if word.lemma_.lower() in harm_reduction_keywords:
                        if date == "2019":
                            print(date, article["_source"]["title"]) 
                        for country in article["_source"]["countries"]:
                            add_to_countries(country["country_name"],date)


HBox(children=(IntProgress(value=0, max=1552), HTML(value='')))

2019 Philip Morris International Gets a Big Boost From IQOS in Q2
2019 Philip Morris International Gets a Big Boost From IQOS in Q2
2019 Philip Morris International Gets a Big Boost From IQOS in Q2
2019 Philip Morris International Gets a Big Boost From IQOS in Q2
2019 Spotlighting IQOS After A Strong Quarter At Philip Morris
2019 Spotlighting IQOS After A Strong Quarter At Philip Morris
2019 Ubervape IQOS
2019 Ubervape IQOS
2019 Ubervape IQOS
2019 Ubervape IQOS
2019 Ubervape IQOS
2019 Ubervape IQOS
2019 IQOS – This Changes Everything
2019 IQOS User's Starter Guide
2019 IQOS User's Starter Guide
2019 IQOS User's Starter Guide
2019 IQOS User's Starter Guide
2019 IQOS User's Starter Guide
2019 IQOS User's Starter Guide
2019 IQOS User's Starter Guide
2019 IQOS boosts Philip Morris International's quarterly profit, revenue. Stock jumps
2019 IQOS boosts Philip Morris International's quarterly profit, revenue. Stock jumps
2019 IQOS HEETS Amber
2019 IQOS HEETS Amber
2019 IQOS HEETS Amber
2019 

2019 Philip Morris launches first Africa store to spark alternative cigarettes demand
2019 Philip Morris launches first Africa store to spark alternative cigarettes demand
2019 Philip Morris launches first Africa store to spark alternative cigarettes demand
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 FDA clears Philip Morris's heat-not-burn IQOS tobacco device for sale in US
2019 IQOS poi

2019 PMI releases new data purporting to show how healthy IQOS is
2019 PMI releases new data purporting to show how healthy IQOS is
2019 PMI releases new data purporting to show how healthy IQOS is
2019 Heated tobacco device given green light by FDA
2019 Heated tobacco device given green light by FDA
2019 Heated tobacco device given green light by FDA
2019 Heated tobacco device given green light by FDA
2019 Heated tobacco device given green light by FDA
2019 Time To Buy Altria - IQOS Will Eventually Be Approved By The FDA
2019 IQOS Drives Philip Morris' Growth In 2018; Outlook Remains Bright
2019 IQOS Drives Philip Morris' Growth In 2018; Outlook Remains Bright
2019 IQOS Drives Philip Morris' Growth In 2018; Outlook Remains Bright
2019 IQOS Drives Philip Morris' Growth In 2018; Outlook Remains Bright
2019 IQOS Drives Philip Morris' Growth In 2018; Outlook Remains Bright
2019 IQOS Drives Philip Morris' Growth In 2018; Outlook Remains Bright
2019 U. S. Backing for Heated Tobacco Products

2019 Why the vaping craze is an opportunity for Philip Morris
2019 Why the vaping craze is an opportunity for Philip Morris
2019 Why the vaping craze is an opportunity for Philip Morris
2019 Why the vaping craze is an opportunity for Philip Morris
2019 Why the vaping craze is an opportunity for Philip Morris
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Philip Morris needs to kill smoking to survive
2019 Why Philip Morris needs to kill smoking to survive
2019 Why Philip Morris needs to kill smoking to survive
2019 Why Philip Morris needs to kill smoking to survive
2019 Why Philip Morris needs to kill smoking to surviv

...

2019 Dangers of youth vaping, juuling
2019 Dangers of youth vaping, juuling
2019 Dangers of youth vaping, juuling
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii eyes flavoured e-cigarette ban to curb teen vaping
2019 Hawaii weighs nation's first statewide ban on e-cig flavours
2019 Hawaii weighs nation's first statewide ban on e-cig flavours
2019 Hawaii weighs nation's first statewide ban on e-cig flavours
2019 Hawaii weighs nation's first statewide ban on e-cig flavours
2019 Hawaii weighs nation's first statewide ban on e-cig flavours
2019 Hawaii weighs nation's first statewide ban on e-cig flavours
2019 Hawaii weighs nation's first statewide ban o

2019 Banning flavored tobacco and vaping products does more harm than good
2019 Banning flavored tobacco and vaping products does more harm than good
2019 Banning flavored tobacco and vaping products does more harm than good
2019 Banning flavored tobacco and vaping products does more harm than good
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping by young people remains a burning issue among health experts
2019 Vaping on the rise, but risks are not known
2019 Vaping on the rise, but risks are not known
2019 Vaping on the rise, but risks are not known
2019 Vaping on the rise, but risks are not

2019 WHAT ARE NICOTINE SALTS AND WHAT TO KNOW WHEN USING THEM.
2019 WHAT ARE NICOTINE SALTS AND WHAT TO KNOW WHEN USING THEM.
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019 Experts at the 6th Global Forum on Nicotine underline the Need for Harm Reduction
2019

2019 Nic Salt E-Liquid from £2.49
2019 Nic Salt E-Liquid from £2.49
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Clearing up the issue of on-campus vaping – HS Insider
2019 Vaping twice better for quitting smoking than patches, gum – study
2019 Vaping twice better for quitting smoking than patches, gum – study
2019 Vaping twice better for quitting smoking than patches, gum – study
2019 Vaping twice better for quitting smoking than patches, gum – study
2019 Vaping twice better for quitting smoking than patches, gum – study
2019 Vaping twice better for qui

2019 Rendez-vous with Louise Ross
2019 Rendez-vous with Louise Ross
2019 Rendez-vous with Louise Ross
2019 Rendez-vous with Louise Ross
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Axios Science
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealand
2019 Health New Zealan

## Convert to Dataframe

In [21]:
vmax= 0

temp = {}
for i, (key, year_dict) in enumerate(harm_reduction_by_country.items()):
    temp[i] = [key]
    for year in year_dict:
        temp[i].append(year_dict[year])
        if year_dict[year] > vmax:
            vmax = year_dict[year]
        
data = pd.DataFrame.from_dict(temp,orient="index")
temp = ["country_code"]
for year in range(1990,2020):
    temp.append(str(year))

data.columns = temp
    
world = gpd.read_file("Geomap/ne_110m_admin_0_countries.shp")[['ADMIN', 'ADM0_A3', 'geometry']]
#world = world.drop(world.index[159]) #drop antartica
world.columns = ['country', 'country_code', 'geometry']

merged = world.merge(data, left_on = 'country_code', right_on = 'country_code')

## Plot

In [24]:
def f(year):
    fig,ax = plt.subplots((1),figsize=(20,10))
    merged.plot(column = str(year),ax=ax,legend=True, vmin=0, vmax=vmax)
    ax.title.set_text('Harm Reduction Globally: '+ str(year))
    
interact(f, year = (2012,2018))



interactive(children=(IntSlider(value=2015, description='year', max=2018, min=2012), Output()), _dom_classes=('widget-interact',))

<function __main__.f>